In [6]:
import csv
import requests
from bs4 import BeautifulSoup

def get_market_csv(each_url, name):
    url = 'https://kabuoji3.com/stock/' + str(each_url) + '/'
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0"}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text.encode(resp.encoding), "html5lib")
    table = soup.findAll("table",{"class":"stock_table"})

    all_rows = []
    for item in table:
        rows = item.findAll("tr")
        del rows[0]
        all_rows.append(rows)

    csvFile = open(name + '.csv', 'wt', newline = '', encoding = 'utf-8')
    writer = csv.writer(csvFile)

    try:
        writer.writerow(['Date','Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'])
        for rows in all_rows:
            for row in rows:
                csvRow = []
                for cell in row.findAll(['td', 'th']):
                    csvRow.append(cell.get_text())
                writer.writerow(csvRow)
    finally:
        csvFile.close()

In [7]:
markets = {
    'samsun200': 1313,
    'shanghai50': 1309,
}
xlist = [
    'diff_samsun200',
    'diff_shanghai50',
]

In [8]:
get_market_csv(1333, 'maruhanichiro')
for name, code in markets.items():
    get_market_csv(code, name)

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick2_ohlc, volume_overlay
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

/opt/conda/lib/python3.6/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [10]:
df_main = pd.read_csv( 'maruhanichiro.csv', parse_dates=True)
dfs = {}
for name in markets.keys():
    dfs[name] = pd.read_csv(name + '.csv', parse_dates=True)

for name, df in dfs.items():
    dates = []
    closeis = []
    yesterday_close = 0
    
    for d in df_main["Date"]:
        date = df.loc[(df.Date == d), "Date"]
        dates.append(d)
        
        close = df.loc[(df.Date == d), "Close"]
        if date.size == 1:
            if str(close.values[0]) != str("nan"):
                yesterday_close = close.values[0]
                closeis.append(close.values[0])	
            else:
                closeis.append(yesterday_close)
        else:
            closeis.append(yesterday_close)
            
    for close in closeis:
        if close != 0:
            yesterday_close = close
            break
            
    for index, close in enumerate(closeis):
        if close == 0:
            closeis[index] = yesterday_close
                
    df2 = pd.DataFrame({"Date" : dates, "Close_" + str(name) : closeis})
    df_main = pd.merge(df_main, df2, how='outer')
    df_main["diff_" + str(name)] = (df_main["Close_" + str(name)] / df_main["Close_" + str(name)].shift(-1)) - 1
df_main.head(20)

,Date,Open,High,Low,Close,Volume,Adj Close,Close_samsun200,diff_samsun200,Close_shanghai50,diff_shanghai50
0,2018-03-16,3280,3300,3265,3275,122000,3275,3280,0.000000,33450,0.002999
1,2018-03-15,3310,3315,3240,3270,172300,3270,3280,0.000000,33350,0.000000
2,2018-03-14,3280,3325,3275,3315,146600,3315,3280,0.000000,33350,-0.007440
3,2018-03-13,3245,3285,3235,3270,112400,3270,3280,0.013910,33600,-0.007386
4,2018-03-12,3275,3280,3230,3250,141800,3250,3235,0.000000,33850,0.004451
5,2018-03-09,3290,3335,3255,3260,317400,3260,3235,0.000000,33700,0.019667
6,2018-03-08,3275,3295,3255,3275,201200,3275,3235,-0.016717,33050,-0.006015
7,2018-03-07,3290,3330,3275,3300,216400,3300,3290,0.084020,33250,-0.002999
8,2018-03-06,3230,3305,3230,3285,294300,3285,3035,-0.014610,33350,0.004518
9,2018-03-05,3165,3210,3165,3200,174300,3200,3080,-0.012821,33200,-0.029240


In [11]:
df_train = df_main.iloc[1:len(df_main)-1]
df_test = df_train.iloc[len(df_train)-1:len(df_train)]
df_test

,Date,Open,High,Low,Close,Volume,Adj Close,Close_samsun200,diff_samsun200,Close_shanghai50,diff_shanghai50
298,2016-12-29,3155,3170,3125,3140,160600,3140,2591,0.0,28700,0.003146


In [12]:
x_train = []
y_train = []
for s in range(0, len(df_train) - 1):
    #print("x_train : ", df_train["Date"].iloc[s])
    #print("y_train : ", df_train["Date"].iloc[s + 1])
    x_train.append(df_train[xlist].iloc[s])
    
    if df_train["Close"].iloc[s + 1] > df_train["Close"].iloc[s]:
        y_train.append(1)
    else:
        y_train.append(-1)

rf = RandomForestClassifier(n_estimators=len(x_train), random_state=0)
rf.fit(x_train, y_train)

test_x = df_test[xlist].iloc[0]
test_y = rf.predict(test_x.values.reshape(1, -1))

print("result : ", test_y[0])

result :  1


In [13]:
def show_market(df):
    fig = plt.figure(figsize=(50, 9))
    ax = plt.subplot(1, 1, 1)

    candlestick2_ohlc(ax, df["Open"], df["High"], df["Low"], df["Adj Close"], width=0.9, colorup="b", colordown="r")

In [14]:
#df_main = pd.read_csv( 'maruhanichiro.csv', parse_dates=True)
#show_market(df_main)
#for df in dfs.values():
#    show_market(df)

In [18]:
import indicators as ind


In [36]:
df_main = ext_ohlc(df_main)
FastMA = ind.iMA(df_main, 10)
SlowMA = ind.iMA(df_main, 30)
BuyEntry = ((FastMA > SlowMA) & (FastMA.shift() <= SlowMA.shift())).values
SellEntry = ((FastMA < SlowMA) & (FastMA.shift() >= SlowMA.shift())).values
BuyExit = SellEntry.copy()
SellExit = BuyEntry.copy()

In [37]:
from pandas_highcharts.display import display_charts
df_chart = pd.DataFrame({'Close': df_main['Close'], 'FastMA': FastMA, 'SlowMA': SlowMA})
display_charts(df_chart, chart_type="stock", title="MA cross", figsize=(640,480), grid=True)

In [42]:
def Backtest(ohlc, BuyEntry, SellEntry, BuyExit, SellExit, lots=100):
    Open = ohlc['Open'].values #始値
    Lots = lots*1
    N = len(ohlc) #FXデータのサイズ
    BuyExit[N-2] = SellExit[N-2] = True #最後に強制エグジット
    BuyPrice = SellPrice = 0.0 # 売買価格

    LongTrade = np.zeros(N) # 買いトレード情報
    ShortTrade = np.zeros(N) # 売りトレード情報

    LongPL = np.zeros(N) # 買いポジションの損益
    ShortPL = np.zeros(N) # 売りポジションの損益

    for i in range(1,N):
        if BuyEntry[i-1] and BuyPrice == 0: #買いエントリーシグナル
            BuyPrice = Open[i]
            LongTrade[i] = BuyPrice #買いポジションオープン
        elif BuyExit[i-1] and BuyPrice != 0: #買いエグジットシグナル
            ClosePrice = Open[i]
            LongTrade[i] = -ClosePrice #買いポジションクローズ
            LongPL[i] = (ClosePrice-BuyPrice)*Lots #損益確定
            BuyPrice = 0

        if SellEntry[i-1] and SellPrice == 0: #売りエントリーシグナル
            SellPrice = Open[i]
            ShortTrade[i] = SellPrice #売りポジションオープン
        elif SellExit[i-1] and SellPrice != 0: #売りエグジットシグナル
            ClosePrice = Open[i]
            ShortTrade[i] = -ClosePrice #売りポジションクローズ
            ShortPL[i] = (SellPrice-ClosePrice)*Lots #損益確定
            SellPrice = 0

    return pd.DataFrame({'Long':LongTrade, 'Short':ShortTrade}, index=ohlc.index),\
            pd.DataFrame({'Long':LongPL, 'Short':ShortPL}, index=ohlc.index)

Trade, PL = Backtest(df_main, BuyEntry, SellEntry, BuyExit, SellExit)

In [43]:
def BacktestReport(Trade, PL):
    LongPL = PL['Long']
    LongTrades = np.count_nonzero(Trade['Long'])//2
    LongWinTrades = np.count_nonzero(LongPL.clip_lower(0))
    LongLoseTrades = np.count_nonzero(LongPL.clip_upper(0))
    print('買いトレード数 =', LongTrades)
    print('勝トレード数 =', LongWinTrades)
    print('最大勝トレード =', LongPL.max())
    print('平均勝トレード =', round(LongPL.clip_lower(0).sum()/LongWinTrades, 2))
    print('負トレード数 =', LongLoseTrades)
    print('最大負トレード =', LongPL.min())
    print('平均負トレード =', round(LongPL.clip_upper(0).sum()/LongLoseTrades, 2))
    print('勝率 =', round(LongWinTrades/LongTrades*100, 2), '%\n')

    ShortPL = PL['Short']
    ShortTrades = np.count_nonzero(Trade['Short'])//2
    ShortWinTrades = np.count_nonzero(ShortPL.clip_lower(0))
    ShortLoseTrades = np.count_nonzero(ShortPL.clip_upper(0))
    print('売りトレード数 =', ShortTrades)
    print('勝トレード数 =', ShortWinTrades)
    print('最大勝トレード =', ShortPL.max())
    print('平均勝トレード =', round(ShortPL.clip_lower(0).sum()/ShortWinTrades, 2))
    print('負トレード数 =', ShortLoseTrades)
    print('最大負トレード =', ShortPL.min())
    print('平均負トレード =', round(ShortPL.clip_upper(0).sum()/ShortLoseTrades, 2))
    print('勝率 =', round(ShortWinTrades/ShortTrades*100, 2), '%\n')

    Trades = LongTrades + ShortTrades
    WinTrades = LongWinTrades+ShortWinTrades
    LoseTrades = LongLoseTrades+ShortLoseTrades
    print('総トレード数 =', Trades)
    print('勝トレード数 =', WinTrades)
    print('最大勝トレード =', max(LongPL.max(), ShortPL.max()))
    print('平均勝トレード =', round((LongPL.clip_lower(0).sum()+ShortPL.clip_lower(0).sum())/WinTrades, 2))
    print('負トレード数 =', LoseTrades)
    print('最大負トレード =', min(LongPL.min(), ShortPL.min()))
    print('平均負トレード =', round((LongPL.clip_upper(0).sum()+ShortPL.clip_upper(0).sum())/LoseTrades, 2))
    print('勝率 =', round(WinTrades/Trades*100, 2), '%\n')

    GrossProfit = LongPL.clip_lower(0).sum()+ShortPL.clip_lower(0).sum()
    GrossLoss = LongPL.clip_upper(0).sum()+ShortPL.clip_upper(0).sum()
    Profit = GrossProfit+GrossLoss
    Equity = (LongPL+ShortPL).cumsum()
    MDD = (Equity.cummax()-Equity).max()
    print('総利益 =', round(GrossProfit, 2))
    print('総損失 =', round(GrossLoss, 2))
    print('総損益 =', round(Profit, 2))
    print('プロフィットファクター =', round(-GrossProfit/GrossLoss, 2))
    print('平均損益 =', round(Profit/Trades, 2))
    print('最大ドローダウン =', round(MDD, 2))
    print('リカバリーファクター =', round(Profit/MDD, 2))
    return Equity

Equity = BacktestReport(Trade, PL)

買いトレード数 = 8
勝トレード数 = 3
最大勝トレード = 8000.0
平均勝トレード = 5500.0
負トレード数 = 5
最大負トレード = -15000.0
平均負トレード = -8200.0
勝率 = 37.5 %

売りトレード数 = 8
勝トレード数 = 4
最大勝トレード = 18000.0
平均勝トレード = 10250.0
負トレード数 = 4
最大負トレード = -23500.0
平均負トレード = -15250.0
勝率 = 50.0 %

総トレード数 = 16
勝トレード数 = 7
最大勝トレード = 18000.0
平均勝トレード = 8214.29
負トレード数 = 9
最大負トレード = -23500.0
平均負トレード = -11333.33
勝率 = 43.75 %

総利益 = 57500.0
総損失 = -102000.0
総損益 = -44500.0
プロフィットファクター = 0.56
平均損益 = -2781.25
最大ドローダウン = 72000.0
リカバリーファクター = -0.62


In [45]:
Initial = 3000000 # 初期資産
display_charts(pd.DataFrame({'Equity':Equity+Initial}), chart_type="stock", title="資産曲線", figsize=(640,480), grid=True)

In [47]:
def PositionLine(trade):
    PosPeriod = 0 #ポジションの期間
    Position = False #ポジションの有無
    Line = trade.copy()
    for i in range(len(Line)):
        if trade[i] > 0: Position = True 
        elif Position: PosPeriod += 1 # ポジションの期間をカウント
        if trade[i] < 0:
            if PosPeriod > 0:
                Line[i] = -trade[i]
                diff = (Line[i]-Line[i-PosPeriod])/PosPeriod
                for j in range(i-1, i-PosPeriod, -1):
                    Line[j] = Line[j+1]-diff # ポジションの期間を補間
                PosPeriod = 0
                Position = False
        if trade[i] == 0 and not Position: Line[i] = 'NaN'
    return Line

df = pd.DataFrame({'Open': df_main['Open'],
                   'Long': PositionLine(Trade['Long'].values),
                   'Short': PositionLine(Trade['Short'].values)})
display_charts(df, chart_type="stock", title="トレードチャート", figsize=(640,480), grid=True)